In [1]:
import pandas as pd
import os
import subprocess 

In [2]:
# wza_kendalltau_results_bio1.csv'
# wza_results_lfmm_bio1.csv

In [3]:
tair = pd.read_csv('../key_files/TAIR10_GFF3_genes_transposons_formatted4topr_w_blocks.csv')

In [4]:
import pickle
dict_blocks = '../key_files/blocks_snpsid_dict.pkl'

with open(dict_blocks, 'rb') as file:
    dict_blocks = pickle.load(file)

reverse_mapping = {item: key for key, values in dict_blocks.items() for item in values}

In [5]:
biovar = 'bio18'

In [88]:
sign_blocks_union_first_last_gen = pd.read_csv(f'top_hits_first_last_gen_{biovar}.csv')
#sign_blocks_union_first_last_gen = pd.read_csv(f'all_intersections_gea_gwas231_last_gen_blocks.csv')

sign_blocks_union_first_last_gen = pd.read_csv(f'sign_blocks_union_last_gen_BH_{biovar}.csv')

In [89]:
#sign_blocks_union_first_last_gen.columns = ['biovar', 'comparision', 'block']

In [91]:
df = sign_blocks_union_first_last_gen.copy()

In [24]:
sign_blocks_union_first_last_gen = full_list.copy()

In [25]:
df = full_list.copy()

In [26]:
bed_file_name = f"top_hits_sign_blocks_union_first_last_gen_BH_final_{biovar}.bed"

In [27]:
blocks_annotated_file = f"top_hits_sign_blocks_union_first_last_gen_blocks_annotated_BH_final_{biovar}.txt" 

In [28]:
gene_ids_file = f'top_hits_sign_blocks_union_first_last_gen_gene_ids_BH_final_{biovar}.txt'

In [29]:
gff_filtered_file = f'top_hits_sign_blocks_union_first_last_gen_BH_final_{biovar}.gff'

In [30]:
nucleotide_seq_sign_genes = f'top_hits_sign_blocks_union_first_last_gen_nucleotide_seq_BH_final_{biovar}.fa'

In [31]:
# Open the BED file for writing
with open(bed_file_name, "w") as bed_file:
    for block in df['block']:
        if block in dict_blocks:
            # Extract SNPs for the block
            snps = dict_blocks[block]
            
            # Assuming the format of SNP is like '1_291' where '1' is the chromosome and '291' is the position
            # We need to find the min and max positions for the block to create the start-end range
            chrom_positions = [snp.split('_') for snp in snps]
            chrom = chrom_positions[0][0]  # The chromosome, assuming all SNPs are from the same chromosome
            
            # Prepend 'Chr' to the chromosome name to match GFF naming convention
            chrom = f"Chr{chrom}"
            
            # Get the minimum and maximum position for the block
            start = min(int(pos) for chrom, pos in chrom_positions)
            end = max(int(pos) for chrom, pos in chrom_positions)
            
            # Write the block information in BED format: chrom, start, end, block name
            bed_file.write(f"{chrom}\t{start}\t{end}\t{block}\n")

print("BED file has been created with 'Chr' prefix in chromosome names.")


BED file has been created with 'Chr' prefix in chromosome names.


In [32]:
bed_file_name

'top_hits_sign_blocks_union_first_last_gen_BH_final_bio18.bed'

In [33]:
## this will, based on the TAIR10_GFF3_genes_transposons cehck the intersectionin between the bed file fo hte singificant blocks and the genes 
gff_file = "../key_files/TAIR10_GFF3_genes_transposons.gff"  # Replace with your actual GFF3 file path

# Construct the command to load the bedtools module and run the intersect command
cmd = f"""
module load bedtools/2.31.1; \
bedtools intersect -wb -a {bed_file_name} -b {gff_file} > {blocks_annotated_file}
"""

# Run the command
try:
    subprocess.run(cmd, shell=True, check=True, executable="/bin/bash")  # Use bash to run the command
    print(f"Intersection complete. Results saved in {blocks_annotated_file}")
except subprocess.CalledProcessError as e:
    print(f"Error running bedtools: {e}")


Intersection complete. Results saved in top_hits_sign_blocks_union_first_last_gen_blocks_annotated_BH_final_bio18.txt


In [34]:
annot = pd.read_csv(blocks_annotated_file, sep='\t', header=None)

In [35]:
annot.columns = [
    'block_chrom', 'block_start', 'block_end', 'block_id',  # From BED file
    'gene_chrom', 'source', 'feature', 'gene_start', 'gene_end',  # From GFF file
    'score', 'strand', 'frame', 'attribute'  # From GFF file
]

In [36]:
annot = annot[annot['attribute'].str.contains('protein_coding_gene')]

In [37]:
annot['gene_id'] = annot['attribute'].str.split(';').str[0].str.replace('ID=', '')

In [38]:
sign_blocks_union_first_last_gen = sign_blocks_union_first_last_gen.merge(annot, left_on = 'block', right_on = 'block_id')

In [40]:
sign_blocks_union_first_last_gen.to_csv(f'top_hits_gene_and_model_sign_blocks_union_first_last_gen_BH_{biovar}.csv')

In [41]:
# Assuming your DataFrame has a column for gene IDs, e.g., 'gene_id'
gene_ids = annot['gene_id'].unique()  # Get unique gene IDs

# Write these gene IDs to a file to use for filtering the GFF3 file
with open(gene_ids_file, 'w') as f:
    for gene_id in gene_ids:
        f.write(f"{gene_id}\n")
        
print("Gene IDs saved to gene_ids.txt")

Gene IDs saved to gene_ids.txt


In [44]:

false_disc = pd.read_csv(f'top_hits_gene_and_model_sign_blocks_union_first_last_gen_BH_{biovar}.csv')

In [46]:
false_disc = false_disc.drop_duplicates(subset=['gene_id'])

In [50]:
from Bio import Entrez

# Provide your email, NCBI requires this for queries
Entrez.email = "tbellg@berkeley.edu"

# Function to retrieve gene information using NCBI Entrez API
def get_gene_info_ncbi(gene_id):
    try:
        # Search for the gene in the NCBI database
        search_handle = Entrez.esearch(db="gene", term=f"{gene_id}[Gene]", retmax=1)
        search_results = Entrez.read(search_handle)
        search_handle.close()
        
        if search_results["IdList"]:
            gene_ncbi_id = search_results["IdList"][0]  # Get the first matching gene ID
            
            # Fetch detailed gene information using the gene ID
            fetch_handle = Entrez.efetch(db="gene", id=gene_ncbi_id, retmode="xml")
            gene_data = Entrez.read(fetch_handle)
            fetch_handle.close()
            
            # Extract relevant information from the gene data
            gene_info = gene_data[0]
            gene_name = gene_info['Entrezgene_gene']['Gene-ref']['Gene-ref_locus']
            gene_desc = gene_info['Entrezgene_summary']
            return {
                "gene_id": gene_id,
                "gene_name": gene_name,
                "description": gene_desc
            }
        else:
            return None
    except Exception as e:
        print(f"Error fetching data for gene {gene_id}: {e}")
        return None

# Initialize an empty list to store gene information and the corresponding block
gene_info_list = []
    # Loop through each gene_id and retrieve information from the API

for index,row in false_disc.iterrows():
    gene_id =  row['gene_id']
    block = row['block']
    print(gene_id)
    gene_info = get_gene_info_ncbi(gene_id.strip())  # Ensure any extra spaces are removed
    if gene_info:
        # Add the gene information and the corresponding block to the list
        gene_info_list.append({
            "block": block,
            "gene_id": gene_info.get('gene_id'),
            "gene_name": gene_info.get('gene_name'),
            "description": gene_info.get('description')
        })

AT2G17640
AT2G17650
Error fetching data for gene AT2G17650: 'Gene-ref_locus'
AT2G17660
Error fetching data for gene AT2G17660: 'Gene-ref_locus'
AT2G17670
Error fetching data for gene AT2G17670: 'Gene-ref_locus'
AT2G17680
Error fetching data for gene AT2G17680: 'Gene-ref_locus'
AT2G17690
AT2G17695
Error fetching data for gene AT2G17695: 'Gene-ref_locus'
AT2G17700
Error fetching data for gene AT2G17700: 'Entrezgene_summary'
AT2G17705
Error fetching data for gene AT2G17705: 'Gene-ref_locus'
AT2G17710
Error fetching data for gene AT2G17710: 'Gene-ref_locus'
AT2G17720
AT2G17630
Error fetching data for gene AT2G17630: 'Gene-ref_locus'
AT2G17723
Error fetching data for gene AT2G17723: 'Gene-ref_locus'
AT3G29350
AT3G29360
AT5G03190
Error fetching data for gene AT5G03190: 'Entrezgene_summary'
AT5G03200
AT5G54130
Error fetching data for gene AT5G54130: 'Gene-ref_locus'
AT5G54140
AT1G23550
AT1G31740
AT2G27030
AT2G14890
AT4G13980
AT4G19230
AT4G39520
Error fetching data for gene AT4G39520: 'Gene-re

In [51]:
df_gene_info = pd.DataFrame(gene_info_list)

In [ ]:
AAE18

In [57]:
df_gene_info[df_gene_info['gene_name'] == 'AAE18']

,block,gene_id,gene_name,description
23,1_2859,AT1G55320,AAE18,Encodes a protein with similarity to acyl acti...


In [124]:
pd.Series(gene_names).unique()

array(['ATSERAT3;1', 'SDC', 'P4H5', 'AHP2', 'UGD2', 'LUL1', 'ILL3',
       'SRO2', 'BGAL15', 'CAM5', 'AGP9', 'AT-HSFA5', 'CYP707A1', 'SK2',
       'ACS12', 'ILL6', 'RGA1', 'PRD1', 'ATCSLB05', 'HSF A4A', 'IOS1',
       'GLIP2', 'SCL33', 'AAE18', 'AGP21', 'GCT', 'DEK1', 'MAP70-1',
       'FKF1', 'GAI', 'CPN10', 'QSOX1', 'GAT1_2.1', 'EER4', 'ARA1',
       'PLDGAMMA1', 'SMO1-1', 'SEC1B', 'WNK9', 'CYSD2', 'CYP71A19',
       'TPS12', 'ABF2', 'THO5', 'NRAMP5', 'NAC089', 'CIPK25', 'CYP71B11',
       'CYP71B12', 'CYP71B13', 'TAF5', 'ZFP3', 'ESE3'], dtype=object)

In [83]:
df_gene_info

,block,gene_id,gene_name,description
0,3_2534,AT3G47500,CDF3,Dof-type zinc finger domain-containing protein...
1,5_724,AT5G24670,EMB2820,A protein coding gene with unknown function. T...
2,5_724,AT5G24770,VSP2,Has acid phosphatase activity dependent on the...
3,5_724,AT5G24780,VSP1,encodes an acid phosphatase similar to soybean...
4,5_724,AT5G24800,BZIP9,Encodes bZIP protein BZO2H2.
...,...,...,...,...
672,5_335,AT5G09870,CESA5,Encodes a cellulose synthase CESA5 that produc...
673,5_335,AT5G09610,PUM21,Encodes a member of the Arabidopsis Pumilio (A...
674,5_335,AT5G09805,IDL3,Similar to Inflorescence deficient in abscissi...
675,5_630,AT5G23720,PHS1,Encodes a protein tyrosine phosphatase Propyza...


In [29]:
false_disc = false_disc.merge(df_gene_info, on = 'gene_id')

In [31]:
false_disc = false_disc.drop_duplicates()

In [65]:
false_disc = false_disc[['biovar', 'comparision','block_id', 'block_chrom', 'block_start',
       'block_end',
       'gene_id',  'gene_name', 'description']]

In [71]:
false_disc = false_disc.drop_duplicates()

In [91]:
false_disc.to_csv(f'top_hits_genes_info_tair10_{biovar}.csv',index=None)